<a href="https://colab.research.google.com/github/Lalit-eng/gpay/blob/main/Gpay_MID_Check_Soundpod_N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyxlsb

import os
import pandas as pd
import sqlite3
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set your folder path
folder_path = '/content/drive/MyDrive/check'
db_path = '/content/merged_data.db'

# Connect to SQLite
conn = sqlite3.connect(db_path)

# Track added files
files_added = 0

for file in os.listdir(folder_path):
    if file.endswith(('.xlsx', '.xls', '.xlsb')):
        file_path = os.path.join(folder_path, file)
        try:
            # Read based on file type
            if file.endswith('.xlsb'):
                df = pd.read_excel(file_path, engine='pyxlsb')
            else:
                df = pd.read_excel(file_path)

            # Normalize column names (lowercase and strip spaces)
            df.columns = df.columns.str.strip().str.lower()

            # Ensure the unique_id column exists (adjust name if needed)
            if 'merchantexternalid' in df.columns:
                df.to_sql('cases', conn, if_exists='append', index=False)
                print(f"✅ Added: {file}")
                files_added += 1
            else:
                print(f"⚠️ Skipped {file}: 'unique_id' column not found.")

        except Exception as e:
            print(f"❌ Failed to read {file}: {e}")

print(f"✅ Total files added: {files_added}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Added: SP Master Feb'25.xlsb
✅ Added: SP Master Mar'25.xlsb
✅ Added: SP Master Feb'25.xlsx
✅ Total files added: 3


In [ ]:
import sqlite3
import pandas as pd

# Define the unique ID to search for
uid = input('Enter merchantexternalid-')

# Reconnect to the DB if needed
conn = sqlite3.connect('/content/merged_data.db')

# Run the SQL query
query = f"SELECT * FROM cases WHERE merchantexternalid = '{uid}'"
result = pd.read_sql_query(query, conn)

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# Display results
if result.empty:
    print("❌ No record found with that unique ID.")
else:
    print("✅ Record found:")
    print(result)  # Explicit print to control output

# Close DB after query
conn.close()
